In [ ]:
"""
Based on:
https://medium.com/@judopro/using-machine-learning-to-programmatically-determine-stock-support-and-resistance-levels-9bb70777cf8e
https://realpython.com/k-means-clustering-python/
"""
from binance_utils import init
from trade_utils import get_data
from machine_learning_utils import get_kmeans_clusters
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

symbol = 'BTCUSDT'
start = '2022-01-01'
end = '2022-07-09'
strategy_interval = '1D'

client = init()
data = get_data(client, symbol, strategy_interval)
#limit by date
data = data[start:end]
#limit by number of candles
#data = data[-60:]

lows = pd.DataFrame(data=data, index=data.index, columns=['LowPrice'])
highs = pd.DataFrame(data=data, index=data.index, columns=['HighPrice'])

low_clusters, lwcss = get_kmeans_clusters(lows)
low_centers = low_clusters.cluster_centers_
low_centers = np.sort(low_centers, axis=0)
print(low_centers)

k = low_centers.size
plt.plot(range(1, 11), lwcss)
plt.scatter(range(1, 11), lwcss)
plt.scatter(k, lwcss[k], marker='X', s = 300, c = 'r')
plt.title('The Elbow Method - Support')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

high_clusters, hwcss = get_kmeans_clusters(highs)
high_centers = high_clusters.cluster_centers_
high_centers = np.sort(high_centers, axis=0)
print(high_centers)

k = high_centers.size
plt.plot(range(1, 11), hwcss)
plt.scatter(range(1, 11), hwcss)
plt.scatter(k, hwcss[k], marker='X', s = 300, c = 'r')
plt.title('The Elbow Method - Support')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

fig = go.Figure(data=go.Candlestick(x=data.index,
                open=data['OpenPrice'],
                high=data['HighPrice'],
                low=data['LowPrice'],
                close=data['ClosePrice']))

for low in low_centers:
    fig.add_hline(y=low[0], 
                  line_dash='dash', 
                  line_color='red', 
                  annotation_text=low[0],
                  annotation_position='bottom right',
                  annotation_font_size=20,
                  exclude_empty_subplots=True)

for high in high_centers[-2:]:
    fig.add_hline(y=high[0], 
                  line_dash='dash', 
                  line_color='green',
                  annotation_text=high[0],
                  annotation_position='top right',
                  annotation_font_size=20,
                  exclude_empty_subplots=True)

fig.update_layout(title='Support - Resistance levels', xaxis_rangeslider_visible=False, width=1400, height=800)
fig.show()